In [10]:
from scipy.io import wavfile as wav
import numpy as np
import scipy
import os
import sys

def band_pass(x,sr,fL,fU):
    x = list(np.array(x))           #converting array to list
    i = 1
    out = []
    while i*sr<len(x):      #sampling rate X time actually i is the time to reach all the samples
        x_ = x[(i-1)*sr:i*sr]   #considering the values of the currently working i only its equal to sr actually
        fft_out = np.fft.fft(x_)        #fast fourier transformation taking from time domain to frequency domain
        fft_out[0:fL] = 0           #FL is the lowest index from which we will start counting the freq
        fft_out[fU:-1] = 0             #FU is highest freaq we will consider for the horn
        wave = np.fft.ifft(fft_out)     #For the portion we wont consider we make it 0 to cut off noise
        out += list(wave.real)            #fft sometimes gives complex number we must only consider the real part
        i += 1
    return np.array(out).astype(np.int16)#converting list to np array of integer

def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def convert_to_hhmmss(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h,24)
    return "%02d:%02d:%02d" % (h, m, s)

def date_time(text,i):
    import re
    var = re.findall(r'\d+',text.split('/')[-1])
    seconds = get_sec(var[3]+':'+var[4]+':'+var[5]) + i
    return var[1]+'/'+var[2]+'/'+var[0]+' '+convert_to_hhmmss(seconds)

    
def dbfft(x, fs, win=None, ref=32768):

    N = len(x)  # Length of input sequence

    if win is None:
        win = np.ones(N)       #making win np array of 1 
    if len(x) != len(win):
            raise ValueError('Signal and window must be of the same length')
    x = x * win         #conversion

    # Calculate real FFT and frequency vector
    sp = np.fft.rfft(x)                 #calculate the fft and give real valued data only calculated in real domain
    freq = np.arange((N / 2) + 1) / (float(N) / fs) #arange function returns a ndarray of evenly spaced integers i the given range by default
    #produces the fft spectrum

    # Scale the magnitude of FFT by window and factor of 2,
    # because we are using half of FFT spectrum.
    s_mag = np.abs(sp) * 2 / np.sum(win)    #scaling the magnitude

    # Convert to dBFS
    s_dbfs = 20 * np.log10(s_mag/ref)

    # Scale from dBFS to dB
    K = 120
    s_db = s_dbfs + K
    return freq, s_db

    #returns the sampling freq and the new numpy array having scale of dbFS

In [14]:
def main():
    filename='Prantika-DGP 38/Sound/Bus_SOUND_2019_09_23_18_05_19_187.wav'
    s, x = wav.read(filename)
    #coverting to mono channel if stereo
    if len(x.shape) == 2:
        x = x.astype(float).sum(axis=1) / 2
    X = band_pass(x, s, 2000, 5001)
    pwd = os.getcwd()       #get the current working directory
   # os.chdir(folder)
    i=1
    f= open("processed_"+os.path.basename(filename)[:-4] + "_0" + ".txt","w+")
    while i*s<len(X):      #sampling rate X time actually i is the time to reach all the samples
        x_ = X[(i-1)*s:i*s]     #considering the values of the currently working i only its equal to sr actually      
       # Take slice
        N = 8000
        win = np.hamming(N)      #https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.hamming.html
                                #we want the array final to have 8000 value.hamming(m):m->int->number of points in the output window
                                #if zero empty array is returned.The output is a normalized numpy array of 8000 values
        freq, s_db = dbfft(x_, s, win)
        t=date_time(filename,i)
        date,time=t.split(" ")

        f.write( date+','+time+","+str(np.max(s_db))+'\n')# logging data timewise and the value
        i += 1
       
    f.close()
    os.chdir(pwd)
    
if __name__== "__main__":
    main()